In [1]:
import numpy as np
import DataLoader,DomainManipulation
import math
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
from hmmlearn.hmm import GMMHMM
import librosa
import time
import mdp
from sklearn.decomposition import PCA

C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\numba\errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
start_time = time.time()

traindir = 'data/train_audio'
targetdir = 'data/birdseperation-sfa-composit'

dirs = os.listdir(traindir)
dirs = dirs[:1]

# for birddir in dirs:
birddir = dirs[0]
files = os.listdir(os.path.join(traindir,birddir))

sfa_components = []
sfa_file_name_order = []
hmm_sfa_corr = []
mfccMap = {}
for fileName in files:
    birdLoc = os.path.join(traindir,birddir,fileName)
    targetLoc = os.path.join(targetdir,birddir,fileName)
    os.makedirs(targetLoc, exist_ok=True)

    try:
        y = DataLoader.readFromFile(birdLoc)
        print("reading file: {}".format(birdLoc))
        mfcc_samples = DomainManipulation.mfcc(y,50).T
        mfccMap[fileName] = mfcc_samples
        
        sfa = mdp.nodes.SFANode(output_dim=1)
        sfa.train(mfcc_samples)
        sfa.stop_training()
        sfa_component = sfa.sf.T
        sfa_samples = sfa(mfcc_samples)

        sfa_file_name_order.append(fileName)
        sfa_components.append(sfa_component.flatten())
    except:
        print("fail")
        continue

sfa_components = np.array(sfa_components)
pca = PCA(n_components=1)
sfa_pca = pca.fit_transform(sfa_components)
sfa_composit = pca.components_[0,:]
sfa_composit = sfa_composit.reshape((len(sfa_composit),1))

reading file: data/train_audio\aldfly\XC134874.mp3
reading file: data/train_audio\aldfly\XC135454.mp3
reading file: data/train_audio\aldfly\XC135455.mp3
reading file: data/train_audio\aldfly\XC135456.mp3
reading file: data/train_audio\aldfly\XC135457.mp3
reading file: data/train_audio\aldfly\XC135459.mp3
reading file: data/train_audio\aldfly\XC135460.mp3
reading file: data/train_audio\aldfly\XC135883.mp3
reading file: data/train_audio\aldfly\XC137570.mp3
reading file: data/train_audio\aldfly\XC138639.mp3
reading file: data/train_audio\aldfly\XC139577.mp3
reading file: data/train_audio\aldfly\XC140298.mp3
reading file: data/train_audio\aldfly\XC142065.mp3
reading file: data/train_audio\aldfly\XC142066.mp3
reading file: data/train_audio\aldfly\XC142067.mp3
reading file: data/train_audio\aldfly\XC142068.mp3
reading file: data/train_audio\aldfly\XC142329.mp3
reading file: data/train_audio\aldfly\XC144672.mp3
reading file: data/train_audio\aldfly\XC144674.mp3
reading file: data/train_audio\

In [3]:
for fileName in files:
    targetLoc = os.path.join(targetdir,birddir,fileName)

    try:
        mfcc_samples = mfccMap[fileName]
        
        sfa_samples = np.matmul(mfcc_samples,sfa_composit)
          
        hmmModel = GMMHMM(n_components=2, n_mix=2)
        ns, blah = sfa_samples.shape
        hmmModel.fit(sfa_samples)
        logP, labels = hmmModel.decode(sfa_samples)
        labels = labels.flatten()
    except:
        print("fail")
        continue

    os.makedirs(os.path.join(targetLoc,"0"), exist_ok=True)
    os.makedirs(os.path.join(targetLoc,"1"), exist_ok=True)
    
    if np.corrcoef(labels, sfa_samples.flatten())[0,1]>0:
        DataLoader.splitByLabels(y, labels, targetLoc)
    else:
        DataLoader.splitByLabels(y, 1-labels, targetLoc)

    plt.subplot(211)
    plt.plot(range(sfa_samples.shape[0]), (sfa_samples[:,0] - np.mean(sfa_samples[:,0]))/(np.std(sfa_samples[:,0])), 'b')
    plt.plot(range(sfa_samples.shape[0]), labels, 'r')

    plt.subplot(212)
    plt.plot(range(sfa_samples.shape[0]), DomainManipulation.rollingMean(y, 0, samples=sfa_samples.shape[0]), 'r')

    plt.savefig(targetLoc+".png")
    plt.clf()
    
sfa_components = np.array(sfa_components)

C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\hmmlearn\utils.py:55: RuntimeWarning: invalid value encountered in subtract
  a -= a_lse


fail
fail
fail
fail
fail
fail
fail


In [4]:
pca.explained_variance_

array([1.19721076e+09])